In [12]:
import os
import time
from selenium.webdriver import Firefox
import pandas as pd

In [13]:
indexes = {'WORLD': '990100',
           'ACWI':  '892400',
           'WORLD VALUE': '105868',
           'ACWI VALUE': '106039'}

In [14]:
indexes = {'WORLD VALUE': '105868',
           'ACWI VALUE': '106039'}

In [15]:
price_levels = {'Gross': 'GRTR',
                'Net': 'NETR'}

In [16]:
[(key, value) for (key, value) in indexes.items()]

[('WORLD VALUE', '105868'), ('ACWI VALUE', '106039')]

In [17]:
driver = Firefox()

In [18]:
driver.implicitly_wait(2.)

In [22]:
for index, code in indexes.items():
    for index_level, price_level in price_levels.items():
        driver.get(f'https://app2.msci.com/products/index-data-search/regional_chart.jsp?currency=USD&priceLevel={price_level}&indexId={code}&indexName={index}')
        time.sleep(5)
        for start_date, end_date in zip(pd.date_range('2001-01-01', 'today', freq='5BAS-JAN').strftime('%b %d, %Y'),
                                        pd.date_range('2005-01-01', 'today', freq='5BA-DEC').union(pd.date_range(pd.to_datetime('today') + pd.tseries.offsets.BMonthEnd(-1, 'D'), periods=1, freq=pd.tseries.offsets.BMonthEnd(-1, 'D'))).strftime('%b %d, %Y')):
            start_date_field = driver.find_element('id', 'startDateFilterShow')
            start_date_field.clear()
            start_date_field.send_keys(start_date)
            driver.find_element('xpath', '/html/body').click()
            end_date_field = driver.find_element('id', 'endDateFilterShow')
            end_date_field.clear()
            end_date_field.send_keys(end_date)
            driver.find_element('xpath', '/html/body').click()
            time.sleep(1)
            driver.find_element('xpath', '/html/body/div[1]/div/section/div/div[4]/div[4]/div[2]/form/div/div/div[8]/div/div/button').click()
            driver.find_element('xpath', '/html/body').click()
            time.sleep(10)
            driver.find_element('xpath', '/html/body/div[1]/div/div/div/div[2]/p[2]/a').click()
            time.sleep(5)
            os.rename('c:\\Users\\leowy\\Downloads\\historyIndex.xls', f'c:\\Users\\leowy\\Downloads\\MSCI {index} USD {index_level} Daily {start_date[-4:]}-{end_date[-4:]}.xls')

In [23]:
driver.quit()